# 11.5
아래에 코드셀을 만들고, 셀에 인접행렬로 표현된 그래프를 인자로 받아 Maximal Spanning Tree를 구하는 하는 함수 maxSpanningTree(...) 를 정의하시오. 함수는 visit 되는 407 페이지에 있는 그림처럼 추가되는 간선을 (x, y, weight) 형태로 출력할 것. 

In [1]:
'''
parent  : 각 정점의 id값을 저장한다. 여기서 순환 그래프를 만들지 않기 위해 각 id값은 방향 그래프 기준으로 저장되어 있다
          즉 A(1)이면 B를 가리키고 있다는 뜻이다. 방향성이 존재하는 이유는 각 집합의 대표번호를 하나로 통일하기 위해서이다.
set_size: 총 집합의 개수를 의미한다

init_set: 정점을 -1로 초기화 한다
find    : 정점 id가 속한 집합에 대표 번호를 반환한다. 예를 들어 B(3) <-> D(4) <->E(-1)이면 정점 B의 대표 번호는 4이다.
          이 함수로 한 정점이 어느 정점하고 연결되어 있는지 알 수 있다
union   : 두 집합을 병합 시킨다. 위에 예를 추가하여 설명하면 EF간선을 추가한다고하면 E(5) <-> F(-1) 이런 식으로 합쳐진다
          병합한 후에 set_size를 하나 줄인다

###### maxSpanningTree ######
모든 간선을 리스트에 넣은 다음 가중치를 기준으로 내림차순으로 정렬한다
pop() 메소드를 이용하여 두 정점의 대표번호를 비교하고 같으면 간선을 추가하지 않고 같지 않으면 간선을 추가한다
'''
parent = []
set_size = 0

def init_set(nSets):  # 집합의 초기화 함수
  global set_size, parent # 전역변수 사용을 위함
  set_size = nSets  # 집합의 개수
  for i in range(nSets):  # 모든 집합에 대해 각각이 고유의 집합(부모가 -1)
    parent.append(-1)

def find(id): # 정점 id가 속한 집합의 대표번호 탐색
  while(parent[id] >= 0): # 부모가 있는 동안(-1이 아닌 동안)
    id = parent[id] # id를 부모 id로 갱신
  return id # 최종 id 반환. 트리의 맨 위 노드의 id임

def union(s1, s2):  # 두 집합을 병합(s1을 s2에 병합시킴)
  global set_size # 전역변수 사용을 위함
  parent[s1] = s2 # s1을 s2에 병합시킴
  set_size = set_size - 1 # 집합의 개수가 줄어 듦

def maxSpanningTree(graph):
  vertex = graph[0]
  adj = graph[1]
  vsize = len(vertex) # 정점의 개수
  init_set(vsize)   # 정점 집합 초기화
  eList = []    # 간선 리스트

  for i in range(vsize-1):  # 모든 간선을 리스트에 넣음
    for j in range(i+1, vsize): 
      if adj[i][j] != None:
        eList.append((i, j, adj[i][j])) # 튜플로 저장

  eList.sort(key = lambda e : e[2])

  edgeAccepted = 0
  while(edgeAccepted < vsize - 1):  # 정점 수 -1개의 간선
    e = eList.pop(-1) # 가장 작은 가중치를 가진 간선
    uset = find(e[0]) # 두 정점이 속한 집합 번호
    vset = find(e[1]) 

    if uset != vset:  # 두 정점이 다른 집합의 원소이면
      print("간선 추가: ({}, {}, {})".format(vertex[e[0]], vertex[e[1]], e[2])) # 간선 추가 출력
      union(uset, vset) # 두 집합을 합함
      edgeAccepted += 1 # 간선이 하나 추가됨

아래 코드셀은 11.5 을 테스트 하는 코드이다. 주어진 데이터를 이용하여 테스트를 실행하시오. 

In [2]:
# 교재 407 페이지 
vertex =   ['A',    'B',    'C',    'D',    'E',    'F',    'G' ]
weight = [ [None,	29,		None,	None,	None,   10,		None],
           [29,	None,	16,		None,	None,	None,	15  ],
           [None,	16,		None,	12,		None,	None,	None],
           [None,	None,   12,		None,	22,		None,	18  ],
           [None,	None,	None,   22,		None,	27,		25  ],
           [10,	None,	None,	None,   27,		None,	None],
           [None,  15,		None,   18,		25,		None,	None]]  

graph = (vertex, weight)

maxSpanningTree(graph)

간선 추가: (A, B, 29)
간선 추가: (E, F, 27)
간선 추가: (E, G, 25)
간선 추가: (D, E, 22)
간선 추가: (B, C, 16)
간선 추가: (B, G, 15)


# 11.6
아래에 코드셀을 만들고, 문제 11.6 의 솔루션을 제공하는 함수 dijkstra_SP_with_path_print(...)을 작성하시오. 교재 428 문제 11.6에 보이는 바와 같이 출력하도록 작성하시오. 

In [9]:
'''
INF: dijkstra알고리즘을 수행하기 전에는 간선이 존재하지 않는다는 의미이고 그 이후에는 연결할 수 없다는 의미이다.

          choose_vertex    : 정점 중에 가중치가 가장 낮은 정점의 위치를 반환하는 함수이다.
dijkstra_SP_with_path_print: shortest_path_dijkstra함수를 통해서 얻은 각 정점까지의 가중치와 경로를 출력하는 함수이다
###### shortest_path_dijkstra ######

가장 짧은 경로를 반환하는 함수이다.
dijkstra 알고리즘의 핵심은 나와 연결된 정점들 중에 한 정점으로 가야 더 빨리 갈 수 있다는 것을 알 수 있다.
예를 들어서 E가 연결된 정점들이 C, D, F라고 하면 C를 통해서 갈 경우 가중치가 10, D는 12, F는 9라고 하면,
F를 통해 어느 정점을 거쳐갈 지는 모르지만, 분명한 것은 F 정점으로 가야지 가장 가중치가 적다는 것을 알 수 있다.
그렇게 각 정점은 A로 갈 때 어느 정점으로 가야 최소 가중치를 가질지의 정보를 가지고 있다.

dist : A에서 한 정점으로 갈 때까지 쌓인 가중치를 저장하고 있다.
path : 각 정점에서 A로 갈 때 바로 앞의 정점의 정보를 저장하고 있다. ex) path[1] = 2이면 1번은 2번의 정점으로 가야지 가장 빠르다(최소 가중치이다)라는 것을 의미한다
found: 지나간 정점은 True, 지나가지 않은 정점은 False라고 한다 
'''
INF = 999

def choose_vertex(dist, found): # 최소 dist 정점을 찾는 함수
  min = INF
  minpos = -1
  for i in range(len(dist)):  # 모든 정점 중에서
    if dist[i] < min and found[i] == False: # 방문하지 않은 최소 dist 정점
      min = dist[i]
      minpos = i
  return minpos # 최소 dist 장점의 인덱스 반환

def shortest_path_dijkstra(vtx, adj, start):
  vsize = len(vtx)  # 정점 수
  # 각 정점까지의 가중치를 의미한다. 자기 자신은 0이고 999는 연결되지 않았다(혹은 연결할 수 없다)는 의미이다.
  dist = list(adj[start]) # dist 배열 생성 및 초기화
  # 한 정점에서 다른 정점으로 이동할 때 이동한 경로를 저장
  path = [start] * vsize  # path 배열 생성 및 초기화
  found = [False] * vsize # found 배열 생성 및 초기화
  found[start] = True # 시작정점: 이미 찾아짐
  dist[start] = 0 # 시작정점의 거리 0
  
  for i in range(vsize): 
    # 최소 정점 순서대로 정점들의 가중치를 갱신한다
    u = choose_vertex(dist, found)  # 최소 dist 정점 u 탐색
    found[u] = True # u는 이제 찾아짐

    for w in range(vsize):  # 모든 정점에 대해
      if not found[w]:  # 아직 찾아지지 않았으면
        if dist[u] + adj[u][w] < dist[w]: # 갱신 조건 검사
          dist[w] = dist[u] + adj[u][w] # dist 갱신
          path[w] = u # 이전 정점 갱신
  
  return path # 찾아진 최단 경로 반환

def dijkstra_SP_with_path_print(start, graph):
  path = shortest_path_dijkstra(graph[0], graph[1], start)  # 시작 정점은 0번 'A'로 선택
  list_path = list()  # 정점이 지나온 길을 역순으로 저장한다.
  dist = 0  # 시점에서 종점까지 가중치를 모두 더한 값이다
  print("Src->Dst\tDist\tPath")
  for end in range(len(vertex)):
    if end != start:
      print("  {}->{}\t\t".format(start, end), end='')  # 시점과 종점 위치 출력
      list_path.append(end)
      while(path[end] != start):
        list_path.append(path[end])
        dist += weight[end][path[end]]
        end = path[end]
      list_path.append(path[end]) # 지나온 정점 저장
      dist += weight[end][path[end]]  # 지아온 정점의 가중치를 합한다
      print(dist, "\t", end="") # 지나온 길의 총 가중치 합 출력

      for v in range(len(list_path)):
        print("{} ".format(list_path.pop()), end="")  # 지나쳐온 정점 순서대로 출력
      print()
      list_path.clear() # 지나온 길 초기화
      dist = 0  # 가중치 합 초기화


아래 코드셀은 11.6 을 테스트 하는 코드이다. 주어진 데이터를 이용하여 테스트를 실행하시오.

In [10]:
vertex =   ['A',    'B',    'C',    'D',    'E',    'F',    'G' 	]
weight = [ [0,	    7,		INF,		INF,		3,      10,		INF	],
           [7,		0,	    4,		10,	    2,	    6,	    INF	],
           [INF,	4,		0,		2,		INF,		INF,		INF	],
           [INF,	10,		2,		0,      11,		9,		4   ],
           [3,	    2,	    INF,   	11,		0,      13,		5   ],
           [10,	6,	    INF,		9,      13,		0,		INF	],
           [INF,   INF,		INF,   	4,		5,		INF,		0   ] ]   

graph = (vertex, weight)
start = 0
dijkstra_SP_with_path_print(start, graph) # modified 12.06

Src->Dst	Dist	Path
  0->1		5 	0 4 1 
  0->2		9 	0 4 1 2 
  0->3		11 	0 4 1 2 3 
  0->4		3 	0 4 
  0->5		10 	0 5 
  0->6		8 	0 4 6 
